Version notes:
- after adding `is_holiday` indicator to feature engineered data
    - see if this feature helps
    - try excluding holiday data

In [1]:
import datetime

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1668799821812_0005,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# this data has been updated to include holiday indicator
last_shop_day = "20221117"
df = spark.read.parquet(f"s3://tvlp-ds-users/kendra-frederick/shopping-grid/engineered-features/parquet/{last_shop_day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- market: string (nullable = true)
 |-- out_departure_date: long (nullable = true)
 |-- shop_date: long (nullable = true)
 |-- fare_rank: integer (nullable = true)
 |-- num_itin: long (nullable = true)
 |-- fare: double (nullable = true)
 |-- out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- solution_counts: long (nullable = true)
 |-- shop_counts: long (nullable = true)
 |-- avg_out_avail_max: double (nullable = true)
 |-- avg_out_avail_low: double (nullable = true)
 |-- outDeptDt_dt: date (nullable = true)
 |-- shopDate_dt: date (nullable = true)
 |-- days_til_dept: integer (nullable = true)
 |-- dept_dt_dow_int: integer (nullable = true)
 |-- dept_dt_dow: string (nullable = true)
 |-- is_holiday: integer (nullable = true)
 |-- fr2_fare: double (nullable = true)
 |-- fr2_out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fr2_avg_out_avail_max: double (nullable = true)
 |-- fr2_avg_out_avail_low: double (

In [9]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['market', 'out_departure_date', 'shop_date', 'fare_rank', 'num_itin', 'fare', 'out_cxrs', 'solution_counts', 'shop_counts', 'avg_out_avail_max', 'avg_out_avail_low', 'outDeptDt_dt', 'shopDate_dt', 'days_til_dept', 'dept_dt_dow_int', 'dept_dt_dow', 'is_holiday', 'fr2_fare', 'fr2_out_cxrs', 'fr2_avg_out_avail_max', 'fr2_avg_out_avail_low', 'fr1_fr2_out_cxrs_same', 'fr1_fr2_out_cxrs_overlap', 'avg_fare_dtd', 'fare_prev_shop_day', 'fr2_fare_prev_shop_day', 'avg_out_avail_max_prev_shop_day', 'avg_out_avail_low_prev_shop_day', 'num_itin_prev_shop_day', 'shop_counts_prev_shop_day', 'solution_counts_prev_shop_day', 'fare_prev_dept_day', 'fare_next_dept_day', 'fare_diff_prev_dept_day', 'fare_diff_next_dept_day', 'trailing_avg_shop_counts', 'trailing_avg_solution_counts', 'trailing_avg_fare', 'trailing_std_fare', 'avg_diff_prev_dept_day', 'avg_diff_next_dept_day', 'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day']

In [8]:
# df.select("market").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14

# define features

In [4]:
target_col = 'fare'

# scale: mean = 0, std = 1
features_to_scale = [
    # fare-based features
    'fare_prev_shop_day',
    'fr2_fare_prev_shop_day', 
    'avg_fare_dtd',
    'fare_prev_dept_day', 'fare_next_dept_day',
    'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day',
    'trailing_avg_fare', 'trailing_std_fare',
                    ]
# norm: [0,1]
features_to_norm = [
    'days_til_dept', # encoded in 'avg_fare_dtd', so exclude?
    'num_itin_prev_shop_day',
    # count-based features - expect some skew in these, so norm instead of scale (?)
    'shop_counts_prev_shop_day', 
    'trailing_avg_shop_counts',
    'solution_counts_prev_shop_day',
    'trailing_avg_solution_counts',
]
categorial_features = [
    'dept_dt_dow_int' 
]
as_is_features = [
    # avail-based features are already norm'd
    'avg_out_avail_max_prev_shop_day',
    'avg_out_avail_low_prev_shop_day',
    # boolean feature(s) - new in version d
    'fr1_fr2_out_cxrs_same',
    'fr1_fr2_out_cxrs_overlap',
    'is_holiday'
]
all_features = features_to_scale + features_to_norm + categorial_features + as_is_features
all_cols_for_modeling = all_features + [target_col]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# prep data

## clip

In [7]:
# data should already be clipped for dtd <= 3
# df.groupBy("days_til_dept").count().orderBy("days_til_dept").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|days_til_dept|count|
+-------------+-----+
|            4|  476|
|            5|  476|
|            6|  476|
|            7|  476|
|            8|  476|
|            9|  476|
|           10|  476|
|           11|  476|
|           12|  476|
|           13|  476|
+-------------+-----+
only showing top 10 rows

based on shop date (given that we took a 3-day trailing average -- the first 2 days won't have valid data)

In [10]:
first_shop_date = datetime.date(2022,10,15)
window_size = 3
shop_cutoff = first_shop_date + datetime.timedelta(days=window_size)
df_clip_shop = df.filter(F.col('shopDate_dt') >= shop_cutoff)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Limit to a certain number out into the future -- such as 120

In [24]:
# dtd_limit = 120
# df_clip_dtd = df_clip_shop.filter(F.col("days_til_dept") <= 120)
# df_clip_dtd.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

45548

This cuts out a lot of data, actually

## dealing with nulls

In [5]:
# for col in all_cols_for_modeling:
#     print(col, df_clip_shop.filter(F.col(col).isNull()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fare_prev_shop_day 415
fr2_fare_prev_shop_day 2509
avg_fare_dtd 48
fare_prev_dept_day 392
fare_next_dept_day 392
est_fare_from_prev_dept_day 392
est_fare_from_next_dept_day 392
trailing_avg_fare 415
trailing_std_fare 415
days_til_dept 0
num_itin_prev_shop_day 415
shop_counts_prev_shop_day 415
trailing_avg_shop_counts 415
solution_counts_prev_shop_day 415
trailing_avg_solution_counts 415
dept_dt_dow_int 0
avg_out_avail_max_prev_shop_day 415
avg_out_avail_low_prev_shop_day 415
fare 0

In [23]:
# col = 'fare_prev_shop_day'
# df_null = df_clip_shop.filter(F.col(col).isNull())
# df_null.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------+---------+--------+-------+--------+---------------+-----------+------------------+------------------+------------+-----------+-------------+---------------+-----------+--------+------------+---------------------+---------------------+---------------------+------------------------+------------------+------------------+----------------------+-------------------------------+-------------------------------+----------------------+-------------------------+-----------------------------+------------------+------------------+-----------------------+-----------------------+------------------------+----------------------------+-----------------+-----------------+----------------------+----------------------+---------------------------+---------------------------+
| market|out_departure_date|shop_date|fare_rank|num_itin|   fare|out_cxrs|solution_counts|shop_counts| avg_out_avail_max| avg_out_avail_low|outDeptDt_dt|shopDate_dt|days_til_dept|dept_dt_dow_int|de

A lot of these have high values for days til dept. Which makes sense. 

Either drop nulls for now, or collapse fare-based features into one or a few, e.g. taking an average that ignores the nulls

In [14]:
# df.count(), df_clip_shop.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(133001, 119216)

In [6]:
# df_no_nulls = df_clip_shop.dropna(subset=all_cols_for_modeling)
# df_no_nulls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

115776

In [26]:
# 115776 / 119216

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9711448127768085

Drop for now. Revisit later

In [11]:
df_no_nulls = df_clip_shop.dropna(subset=all_cols_for_modeling)
# df_no_nulls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Train-test split

In [12]:
train, test = df_no_nulls.randomSplit([0.8, 0.2], seed=19)
# train.count(), test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
train_shuffled = train.orderBy(F.rand())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_no_holidays = df_no_nulls.filter(F.col("is_holiday") == 0)
train_no_hol, test_no_hol = df_no_holidays.randomSplit([0.8, 0.2], seed=19)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-53:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# modeling!

## import things

In [15]:
from pyspark.ml.feature import (
    MinMaxScaler, StandardScaler, OneHotEncoderEstimator,
    VectorAssembler, VectorIndexer
)
from pyspark.ml.regression import (
    LinearRegression, RandomForestRegressor
)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## eval code

In [16]:
def calc_abs_pct_err(df, pred_col="prediction"):
    df = (df
           .withColumn("pred_err", F.col("fare") - F.col(pred_col))
           .withColumn("abs_pred_err", F.abs(F.col("pred_err")))
           .withColumn("abs_pct_err", F.col("abs_pred_err") / F.col("fare"))
          )
    print(df.select(F.mean("abs_pct_err").alias("mape")).show())
    # err_by_market = df.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape")
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define Pipelines 

Useful refs:
- https://spark.apache.org/docs/2.4.8/api/python/pyspark.ml.html
    - For a specific algo, see `setParams` method. This enumerates the parameters accepted.

### Linear Regression

In [17]:
ohe = OneHotEncoderEstimator(
    inputCols=categorial_features,
    outputCols=['dept_dow_enc']
)

scale_assembler = VectorAssembler(
    inputCols=features_to_scale,
    outputCol="features_to_scale"
)

scaler = StandardScaler(
    inputCol=scale_assembler.getOutputCol(),
    outputCol="scaled_features"
)

norm_assembler = VectorAssembler(
    inputCols=features_to_norm,
    outputCol="features_to_norm"
)

normer = MinMaxScaler(
    inputCol=norm_assembler.getOutputCol(),
    outputCol="normed_features"
)

# Note: if shuffling data doesn't work, add a VectorSizeHint here
# https://spark.apache.org/docs/2.4.8/ml-features.html#vectorsizehint
# I think this is what's going on: https://www.matthewslipper.com/2017/10/01/watch-out-vectorassembler-incompatible-spark-streaming.html
final_assembler = VectorAssembler(
    inputCols=[*ohe.getOutputCols(), 
               scaler.getOutputCol(),
               normer.getOutputCol()
              ] + as_is_features,
    outputCol='features'
)

lin_regression_model = LinearRegression(
    #  elasticNetParam corresponds to α (ratio) and regParam corresponds to λ (weight).
    labelCol="fare",
    maxIter=20,
    # from tuning
    regParam=1.0,
    elasticNetParam=0.01
)

lin_pipeline = Pipeline(stages=[
    scale_assembler,
    norm_assembler,
    ohe, 
    scaler,
    normer,
    final_assembler,
    lin_regression_model
])

# lin_model = lin_pipeline.fit(train_shuffled)
# test_tx = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## tuning code

In [ ]:
# LINEAR REGRESSION

# param_grid = (ParamGridBuilder()
#               # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
#               # note alpha is the opposite of sklearn's defintion
#               .addGrid(regression_model.elasticNetParam, [0.01, 0.1, 0.5, 0.9, 0.99])
#               .addGrid(regression_model.regParam, [0.01, 0.1, 1, 10, 100])
#               .build()
#              )

# cv = CrossValidator(estimator=pipeline,
#                     estimatorParamMaps=param_grid,
#                     evaluator=RegressionEvaluator(labelCol="fare"),
#                     numFolds=3,
#                     parallelism=5
#                    )

# cv_model = cv.fit(train)

# pred_cv = cv_model.transform(test)

# cv_results = list(zip(
#     cv_model.getEstimatorParamMaps(), 
#     cv_model.avgMetrics
# ))

# cv_results.sort(key=lambda x: x[1])
# best_result = cv_results[0]
# best_metric = best_result[1]
# best_param_dict = best_result[0]
# params = [x.name for x in list(best_param_dict.keys())]
# param_vals = best_param_dict.values()
# best_params = dict(zip(params, param_vals))
# best_params, best_metric

In [ ]:
# RANDOM FOREST

# param_grid = (ParamGridBuilder()
#               # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
#               # note alpha is the opposite of sklearn's defintion
#               .addGrid(regression_model.numTrees, [50, 100, 200])
#               .addGrid(regression_model.maxDepth, [3, 5, 7, 10])
#               .build()
#              )

# cv = CrossValidator(estimator=rf_pipeline,
#                     estimatorParamMaps=param_grid,
#                     evaluator=RegressionEvaluator(labelCol="fare"),
#                     numFolds=3,
#                     parallelism=5
#                    )

# cv_model = cv.fit(train)
# pred_cv = cv_model.transform(test)
# pred_cv = calc_abs_pct_err(pred_cv)

# cv_results = list(zip(
#     cv_model.getEstimatorParamMaps(), 
#     cv_model.avgMetrics
# ))

# cv_results.sort(key=lambda x: x[1])
# best_result = cv_results[0]
# best_metric = best_result[1]
# best_param_dict = best_result[0]
# params = [x.name for x in list(best_param_dict.keys())]
# param_vals = best_param_dict.values()
# best_params = dict(zip(params, param_vals))
# best_params, best_metric

## split out by market

In [ ]:
# temp = df.select("market").distinct().collect()
# market_list = [x['market'] for x in temp]

## TODO: capture results in a dict
# for market in market_list:
#     print(market)
#     market_model = rf_pipeline.fit(train.filter(F.col("market") == market))
#     market_pred_df = market_model.transform(test.filter(F.col("market") == market))
#     market_pred_df = calc_abs_pct_err(market_pred_df)
#     print("")

Note: this was only marginally better than training one model on the whole dataset

# Gradient Boosing

In [19]:
from pyspark.ml.regression import GBTRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

gb_regression_model = GBTRegressor(
    labelCol="fare",
    # maxIter=20,
    # numTrees=100,
    stepSize=0.1,
)

gb_pipeline = Pipeline(stages=[
    raw_feature_assembler,
    gb_regression_model
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## tune

In [27]:
# GBT

gb_param_grid = (ParamGridBuilder()
              # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
              # note alpha is the opposite of sklearn's defintion
              .addGrid(gb_regression_model.maxIter, [10, 20, 50])
              # .addGrid(regression_model.numTrees, [50, 100, 200])
              # .addGrid(gb_regression_model.stepSize, [0.01, 0.05, 0.1, 0.25])
              .addGrid(gb_regression_model.maxDepth, [3, 5, 7, 10])
              .addGrid(gb_regression_model.lossType, ['squared', 'absolute'])
              .build()
             )

cv = CrossValidator(estimator=gb_pipeline,
                    estimatorParamMaps=gb_param_grid,
                    evaluator=RegressionEvaluator(labelCol="fare"),
                    numFolds=3,
                    parallelism=5
                   )

cv_model = cv.fit(train)
pred_cv = cv_model.transform(test)
pred_cv = calc_abs_pct_err(pred_cv)

cv_results = list(zip(
    cv_model.getEstimatorParamMaps(), 
    cv_model.avgMetrics
))

cv_results.sort(key=lambda x: x[1])
best_result = cv_results[0]
best_metric = best_result[1]
best_param_dict = best_result[0]
params = [x.name for x in list(best_param_dict.keys())]
param_vals = best_param_dict.values()
best_params = dict(zip(params, param_vals))
best_params, best_metric

VBox()

Exception in thread cell_monitor-27:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|               mape|
+-------------------+
|0.14425328413773864|
+-------------------+

None
({'maxIter': 50, 'maxDepth': 5, 'lossType': 'squared'}, 117.03455006389265)

In [46]:
pred_cv.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-46:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+--------------------+
| market|                mape|
+-------+--------------------+
|LHR-EWR|0.046982340200654574|
|LHR-JFK| 0.05928787072118872|
|LHR-LAX| 0.09863349562329513|
|EWR-LHR| 0.09981652787013846|
|JFK-LHR| 0.10194048171014869|
|EWR-CDG| 0.10334854786042848|
|LAX-EWR| 0.11075375305733877|
|LAX-JFK| 0.12030932259329453|
|JFK-LAX| 0.12211097365267493|
|ATL-EWR| 0.14210145290178597|
|LGA-MIA|  0.2095614667697796|
|SFO-LAX| 0.21965693629901145|
|LAX-SFO| 0.22010284498942057|
|OAK-LAS| 0.43575656178084005|
+-------+--------------------+

In [ ]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

gb_regression_model = GBTRegressor(
    labelCol="fare",
    # maxIter=20,
    # numTrees=100,
    stepSize=0.1,
)

gb_pipeline = Pipeline(stages=[
    raw_feature_assembler,
    gb_regression_model
])

## feature importances

In [34]:
best_pipeline = cv_model.bestModel
va = best_pipeline.stages[-2]
gb_model = best_pipeline.stages[-1]

feat_imps = list(zip(va.getInputCols(), gb_model.featureImportances))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-34:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [35]:
feat_imps

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('fare_prev_shop_day', 0.06474978370814934), ('fr2_fare_prev_shop_day', 0.04157449094670529), ('avg_fare_dtd', 0.07560167409751524), ('fare_prev_dept_day', 0.04227825202159966), ('fare_next_dept_day', 0.09282471660989441), ('est_fare_from_prev_dept_day', 0.04304553821796271), ('est_fare_from_next_dept_day', 0.06759067485383125), ('trailing_avg_fare', 0.07498048384963131), ('trailing_std_fare', 0.05556758704440851), ('days_til_dept', 0.08747845922728532), ('num_itin_prev_shop_day', 0.020599876964054048), ('shop_counts_prev_shop_day', 0.06502197791453675), ('trailing_avg_shop_counts', 0.03873507489771725), ('solution_counts_prev_shop_day', 0.07928650926106635), ('trailing_avg_solution_counts', 0.05915389724315017), ('dept_dt_dow_int', 0.034913385742295674), ('avg_out_avail_max_prev_shop_day', 0.01744789120685597), ('avg_out_avail_low_prev_shop_day', 0.008852428143561054), ('fr1_fr2_out_cxrs_same', 0.019068112091959688), ('fr1_fr2_out_cxrs_overlap', 0.011036169169070686), ('is_holiday', 

Exception in thread cell_monitor-35:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [37]:
feat_imps.sort(key=lambda x: x[1], reverse=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-37:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [38]:
for x in feat_imps:
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('fare_next_dept_day', 0.09282471660989441)
('days_til_dept', 0.08747845922728532)
('solution_counts_prev_shop_day', 0.07928650926106635)
('avg_fare_dtd', 0.07560167409751524)
('trailing_avg_fare', 0.07498048384963131)
('est_fare_from_next_dept_day', 0.06759067485383125)
('shop_counts_prev_shop_day', 0.06502197791453675)
('fare_prev_shop_day', 0.06474978370814934)
('trailing_avg_solution_counts', 0.05915389724315017)
('trailing_std_fare', 0.05556758704440851)
('est_fare_from_prev_dept_day', 0.04304553821796271)
('fare_prev_dept_day', 0.04227825202159966)
('fr2_fare_prev_shop_day', 0.04157449094670529)
('trailing_avg_shop_counts', 0.03873507489771725)
('dept_dt_dow_int', 0.034913385742295674)
('num_itin_prev_shop_day', 0.020599876964054048)
('fr1_fr2_out_cxrs_same', 0.019068112091959688)
('avg_out_avail_max_prev_shop_day', 0.01744789120685597)
('fr1_fr2_out_cxrs_overlap', 0.011036169169070686)
('avg_out_avail_low_prev_shop_day', 0.008852428143561054)
('is_holiday', 0.0001930167887493982

Exception in thread cell_monitor-38:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [41]:
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-41:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [44]:
file_name = 'gb-feat-impt'
xs = range(len(feat_imps))
plt.bar(xs, [x[1] for x in feat_imps])
plt.xticks(xs, [x[0] for x in feat_imps], rotation=90)
plt.ylabel("feature importance")
plt.title("Gradient Boosted Regression")
plt.tight_layout()
plt.savefig(f"/tmp/{file_name}.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-44:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# Random Forest

Best params from tuning were: 100 tree, maxDepth 7

In [20]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

rf_regression_model = RandomForestRegressor(
    labelCol="fare",
    numTrees=100,
    maxDepth=7,
)

rf_pipeline = Pipeline(stages=[
    raw_feature_assembler,
    rf_regression_model
])

In [47]:
rf_model = rf_pipeline.fit(train_shuffled)
pred_rf_df = rf_model.transform(test)
pred_rf_df = calc_abs_pct_err(pred_rf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-47:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13539775922015176|
+-------------------+

None

## feature importance

In [48]:
va = rf_model.stages[-2]
algo = rf_model.stages[-1]

feat_imps = list(zip(va.getInputCols(), algo.featureImportances))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-48:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [49]:
feat_imps.sort(key=lambda x: x[1], reverse=True)
for x in feat_imps:
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('trailing_avg_fare', 0.3209345075359292)
('fr2_fare_prev_shop_day', 0.20100481104338677)
('fare_prev_shop_day', 0.16585044292420031)
('est_fare_from_next_dept_day', 0.08488915451793487)
('est_fare_from_prev_dept_day', 0.07502049700156743)
('fare_prev_dept_day', 0.04241239533988256)
('avg_fare_dtd', 0.03979723530861093)
('fare_next_dept_day', 0.030494199925413176)
('days_til_dept', 0.00811702169174535)
('trailing_avg_solution_counts', 0.006490518883809328)
('solution_counts_prev_shop_day', 0.005015096955370291)
('trailing_avg_shop_counts', 0.0045890973027149775)
('num_itin_prev_shop_day', 0.003851234028443653)
('shop_counts_prev_shop_day', 0.0035870646399046497)
('avg_out_avail_max_prev_shop_day', 0.002516591545239253)
('trailing_std_fare', 0.0021661553708295293)
('fr1_fr2_out_cxrs_overlap', 0.0012488322404018769)
('fr1_fr2_out_cxrs_same', 0.0010185833979618963)
('dept_dt_dow_int', 0.0009114447005432526)
('avg_out_avail_low_prev_shop_day', 5.1640211248415916e-05)
('is_holiday', 3.34754

Exception in thread cell_monitor-49:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Could try fitting to just the most important features

In [51]:
file_name = 'rf-feat-impt'
xs = range(len(feat_imps))
plt.figure(figsize=(8,6))
plt.bar(xs, [x[1] for x in feat_imps])
plt.xticks(xs, [x[0] for x in feat_imps], rotation=90)
plt.ylabel("feature importance")
plt.title("Random Forest Regression")
plt.tight_layout()
plt.savefig(f"/tmp/{file_name}.png")

VBox()

Exception in thread cell_monitor-51:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## fit vs. no holiday dataset

In [54]:
rf_model = rf_pipeline.fit(train_no_hol)

In [55]:
pred_rf_df = rf_model.transform(test_no_hol)
pred_rf_df = calc_abs_pct_err(pred_rf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-55:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13204314670773593|
+-------------------+

None

In [56]:
pred_rf_df.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-56:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+-------------------+
| market|               mape|
+-------+-------------------+
|LHR-EWR|0.03872246779303075|
|LHR-JFK|0.05642332716873052|
|JFK-LHR|0.06878773573750313|
|EWR-LHR| 0.0925381468617531|
|LAX-EWR|0.09848388174706794|
|LAX-JFK|0.10551663265418912|
|JFK-LAX|0.10636445441698905|
|LHR-LAX|0.10795168470952722|
|ATL-EWR|0.12037048626742097|
|EWR-CDG|0.12553206585929064|
|LGA-MIA| 0.1843305877429315|
|LAX-SFO|0.20743189117602162|
|SFO-LAX|0.21310929811278898|
|OAK-LAS| 0.5397299369846683|
+-------+-------------------+

In [ ]:
# va = rf_model.stages[-2]
# algo = rf_model.stages[-1]

# feat_imps = list(zip(va.getInputCols(), algo.featureImportances))

# file_name = 'rf-feat-impt'
# xs = range(len(feat_imps))
# plt.figure(figsize=(8,6))
# plt.bar(xs, [x[1] for x in feat_imps])
# plt.xticks(xs, [x[0] for x in feat_imps], rotation=90)
# plt.ylabel("feature importance")
# plt.title("Random Forest Regression")
# plt.tight_layout()
# plt.savefig(f"/tmp/{file_name}.png")

## feature selection

In [57]:
feat_imps

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('trailing_avg_fare', 0.3209345075359292), ('fr2_fare_prev_shop_day', 0.20100481104338677), ('fare_prev_shop_day', 0.16585044292420031), ('est_fare_from_next_dept_day', 0.08488915451793487), ('est_fare_from_prev_dept_day', 0.07502049700156743), ('fare_prev_dept_day', 0.04241239533988256), ('avg_fare_dtd', 0.03979723530861093), ('fare_next_dept_day', 0.030494199925413176), ('days_til_dept', 0.00811702169174535), ('trailing_avg_solution_counts', 0.006490518883809328), ('solution_counts_prev_shop_day', 0.005015096955370291), ('trailing_avg_shop_counts', 0.0045890973027149775), ('num_itin_prev_shop_day', 0.003851234028443653), ('shop_counts_prev_shop_day', 0.0035870646399046497), ('avg_out_avail_max_prev_shop_day', 0.002516591545239253), ('trailing_std_fare', 0.0021661553708295293), ('fr1_fr2_out_cxrs_overlap', 0.0012488322404018769), ('fr1_fr2_out_cxrs_same', 0.0010185833979618963), ('dept_dt_dow_int', 0.0009114447005432526), ('avg_out_avail_low_prev_shop_day', 5.1640211248415916e-05), (

Exception in thread cell_monitor-57:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [58]:
feat_impt_cutoff = 0.02
top_feature_imps = [x for x in feat_imps if x[1] > feat_impt_cutoff]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-58:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [59]:
for x in top_feature_imps:
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('trailing_avg_fare', 0.3209345075359292)
('fr2_fare_prev_shop_day', 0.20100481104338677)
('fare_prev_shop_day', 0.16585044292420031)
('est_fare_from_next_dept_day', 0.08488915451793487)
('est_fare_from_prev_dept_day', 0.07502049700156743)
('fare_prev_dept_day', 0.04241239533988256)
('avg_fare_dtd', 0.03979723530861093)
('fare_next_dept_day', 0.030494199925413176)

Exception in thread cell_monitor-59:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Note: these are all likely to be similar / correlated, so worth trying PCA on them.

In [60]:
top_features = [x[0] for x in top_feature_imps]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-60:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [61]:
top_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['trailing_avg_fare', 'fr2_fare_prev_shop_day', 'fare_prev_shop_day', 'est_fare_from_next_dept_day', 'est_fare_from_prev_dept_day', 'fare_prev_dept_day', 'avg_fare_dtd', 'fare_next_dept_day']

Exception in thread cell_monitor-61:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [62]:
top_feature_assembler = VectorAssembler(
    inputCols=top_features,
    outputCol='features'
)

rf_regression_model = RandomForestRegressor(
    labelCol="fare",
    numTrees=100,
    maxDepth=7,
)

rf_pipeline = Pipeline(stages=[
    top_feature_assembler,
    rf_regression_model
])
rf_model = rf_pipeline.fit(train_shuffled)
pred_rf_df = rf_model.transform(test)
pred_rf_df = calc_abs_pct_err(pred_rf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-62:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13534445598849199|
+-------------------+

None

Dang. Actually a little worse

Could try re-tuning

Check train pred's for overfitting?

In [63]:
pred_train = rf_model.transform(train_shuffled)
pred_train = calc_abs_pct_err(pred_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-63:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13068924642212273|
+-------------------+

None